<a href="https://colab.research.google.com/github/OlhaYastrebova/goit-ds-hw/blob/main/Hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import zipfile
import pandas as pd
import os

# Шлях до вашого архіву
zip_file_path = '/content/homework.zip'

# Створюємо словник з активностями та шляхами до відповідних директорій в архіві
activity_dirs = {
    'idle': 'data/idle/',
    'walking': 'data/walking/',
    'running': 'data/running/',
    'stairs': 'data/stairs/'
}

# Ініціалізуємо порожній список для зберігання DataFrame для кожної активності
dataframes = []

# Відкриваємо архів
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
# Отримуємо список всіх файлів в архіві
    all_files = zip_ref.namelist()

# Перебираємо всі активності та їхні директорії
    for activity, directory in activity_dirs.items():
# Знаходимо всі файли в поточній директорії
        activity_files = [f for f in all_files if f.startswith(directory) and f.endswith('.csv')]

# Перебираємо кожен файл в директорії
        for file in activity_files:
# Відкриваємо файл і зчитуємо його в DataFrame
            with zip_ref.open(file) as f:
                df = pd.read_csv(f)

# Додаємо стовпець з типом активності
                df['activity'] = activity

# Додаємо DataFrame до списку
                dataframes.append(df)

# Об'єднуємо всі DataFrame в один загальний DataFrame
    combined_df = pd.concat(dataframes, ignore_index=True)

In [18]:
combined_df

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,1.000776,4.616021,8.576031,idle
1,0.718261,4.209007,8.446744,idle
2,-0.909797,-0.282516,9.203311,idle
3,5.099650,0.148441,8.418014,idle
4,1.762132,-0.162806,9.251195,idle
...,...,...,...,...
193855,3.284845,-11.966220,-3.892972,stairs
193856,3.945645,-11.760319,-8.097190,stairs
193857,2.801216,-9.327810,-5.133168,stairs
193858,-1.350330,-3.380613,-0.057461,stairs


In [23]:
def calculate_features(df):
    # Групуємо дані за типом активності
    grouped = df.groupby('activity')

    # Розраховуємо статистики для кожної групи
    features = grouped.agg({
        'accelerometer_X': ['mean', 'std', 'min', 'max', 'median'],
        'accelerometer_Y': ['mean', 'std', 'min', 'max', 'median'],
        'accelerometer_Z': ['mean', 'std', 'min', 'max', 'median']
    })

    # Спрощуємо імена колонок
    features.columns = ['_'.join(col).strip() for col in features.columns.values]

    return features

features_df = calculate_features(combined_df)
features_df


,accelerometer_X_mean,accelerometer_X_std,accelerometer_X_min,accelerometer_X_max,accelerometer_X_median,accelerometer_Y_mean,accelerometer_Y_std,accelerometer_Y_min,accelerometer_Y_max,accelerometer_Y_median,accelerometer_Z_mean,accelerometer_Z_std,accelerometer_Z_min,accelerometer_Z_max,accelerometer_Z_median
activity,,,,,,,,,,,,,,,
idle,0.096767,0.691892,-10.448297,8.135497,0.234632,2.225971,2.821235,-2.595315,9.916783,0.028730,8.973247,1.211058,-0.296881,21.988348,9.749189
running,5.045705,9.952696,-39.188293,39.188293,3.605668,8.083135,12.441402,-39.188293,39.188293,7.967903,1.387356,7.004236,-39.188293,39.188293,0.407014
stairs,0.353135,3.845949,-11.238382,14.164097,0.483629,-9.574353,4.922294,-25.828648,4.994305,-8.944737,-1.841333,5.297376,-38.086960,17.492037,-1.589750
walking,-2.661943,4.648378,-26.815060,20.575770,-2.298433,-9.703702,5.931165,-39.188293,14.441825,-9.974244,-1.126623,6.927679,-39.169140,38.426937,-0.933739


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Розділяємо дані на ознаки та мітки
X = combined_df.drop(columns='activity')
y = combined_df['activity']

# Розділяємо дані на тренувальні та тестові набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабування ознак
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Створюємо та навчаємо модель SVM
model = SVC(kernel='linear', class_weight='balanced', random_state=42)
model.fit(X_train_scaled, y_train)

# Робимо прогнози
y_pred = model.predict(X_test_scaled)

# Оцінюємо модель
print("Точність:", accuracy_score(y_test, y_pred))
print("Звіт про класифікацію:\n", classification_report(y_test, y_pred))


Точність: 0.7265810378623749
Звіт про класифікацію:
               precision    recall  f1-score   support

        idle       0.74      0.98      0.84      6203
     running       0.98      0.70      0.82     20486
      stairs       0.08      0.66      0.14       915
     walking       0.85      0.63      0.73     11168

    accuracy                           0.73     38772
   macro avg       0.66      0.75      0.63     38772
weighted avg       0.88      0.73      0.78     38772



In [30]:
from sklearn.ensemble import RandomForestClassifier

# Створюємо та навчаємо модель Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Робимо прогнози
y_pred = model.predict(X_test_scaled)

# Оцінюємо модель
print("Точність:", accuracy_score(y_test, y_pred))
print("Звіт про класифікацію:\n", classification_report(y_test, y_pred))

Точність: 0.9997936655318271
Звіт про класифікацію:
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6203
     running       1.00      1.00      1.00     20486
      stairs       1.00      1.00      1.00       915
     walking       1.00      1.00      1.00     11168

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772



Random Forest показує набагато кращі результати в усіх метриках (точність, повнота, F1-міра) порівняно з SVM.
Схоже, що модель Random Forest добре справляється з класифікацією всіх активностей, тоді як SVM має проблеми з деякими класами, зокрема, з класом stairs.

Модель Random Forest виглядає як кращий вибір для класифікації активностей на основі наданих даних.